In [2]:
from pydantic import field_validator
from sqlmodel import SQLModel, Session, Field, create_engine,MetaData
from loguru import logger
from datetime import datetime
from sqlalchemy import Column, CHAR, VARCHAR, BOOLEAN, MetaData, Table, ForeignKey

# 定义 user 表
user_table = Table(
    "user",
    SQLModel.metadata,
    Column("id", CHAR(36), primary_key=True),  # 用户唯一标识
    Column("email", VARCHAR(320), nullable=False, unique=True),  # 用户邮箱
    Column("hashed_password", VARCHAR(1024), nullable=False),  # 加密后的密码
    Column("is_active", BOOLEAN, default=True),  # 用户是否活跃
    Column("is_superuser", BOOLEAN, default=False),  # 用户是否为超级用户
    Column("is_verified", BOOLEAN, default=False),  # 用户是否已验证
)
QUESTION_TYPES = [
    "学业成就型抑郁",
    "身心倦怠型抑郁",
    "同伴关系型抑郁",
    "家庭关系型抑郁",
    "网络创伤型抑郁",
    "身心暗示型抑郁",
]


class Quez(SQLModel, table=True):
    id: int = Field(primary_key=True)
    question: str
    question_type: str = Field()

    # 使用 validator 确保 question_type 的值在 QUESTION_TYPES 中
    @field_validator("question_type")
    def check_question_type(cls, v):
        if v not in QUESTION_TYPES:
            raise ValueError(f"question_type must be one of {QUESTION_TYPES}")
        return v


class Answer(SQLModel, table=True):
    id: int = Field(primary_key=True)
    answer: str
    score: int
    quez_id: int = Field(foreign_key="quez.id", index=True)


class UserAnswers(SQLModel, table=True):
    id: int = Field(primary_key=True)
    user_id: str = Field(sa_column=Column("user_id", CHAR(36), ForeignKey("user.id")))
    answer_id: int = Field(foreign_key="answer.id", index=True)
    quez_id: int = Field(foreign_key="quez.id", index=True)
    score: int
    created_at: datetime = Field(default_factory=datetime.now)




InvalidRequestError: Table 'user' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [3]:
engine = create_engine("sqlite:///quez.sqlite")
SQLModel.metadata.create_all(engine)

In [13]:
import tablib
import os

print(os.getcwd())
quz_data = tablib.Dataset(headers=['题目', '选项'])
with open('DST题库.xlsx', 'rb') as f:
    imported_data = tablib.Dataset().load(f.read(), format='xlsx')

d:\技术学习\A-Frame_learn\backend


In [21]:
def get_answers(s:str):
    l=list(filter(lambda x:len(x)>3,s.split('\n')))
    if len(l)!=5:
        print(s.split('\n'))
    else:
        return l
get_answers(imported_data[0][1])

['0', '一切困难都是可以解决的，我可以尝试改变学习方法或寻求他人帮助', '1', '我担心自己的成绩和未来，需要加倍努力', '1', '成绩有波动也是正常的，我已经尽了自己最大的努力', '2', '我不如以前那么优秀了，提高成绩对于我来说太难了，不知道怎么办', '3', '我以前的“优秀”都是假的，现在努力也是徒劳，未来变得黯淡无光']
['一切困难都是可以解决的，我可以尝试改变学习方法或寻求他人帮助', '我担心自己的成绩和未来，需要加倍努力', '成绩有波动也是正常的，我已经尽了自己最大的努力', '我不如以前那么优秀了，提高成绩对于我来说太难了，不知道怎么办', '我以前的“优秀”都是假的，现在努力也是徒劳，未来变得黯淡无光']


In [15]:
session=Session(engine)
quez_type_range=[(0,9),(10,13),(14,18),(19,27),(28,33),(34,39)]
for i in range(6):
    quez_type=QUESTION_TYPES[i]
    for j in range(quez_type_range[i]):
        question_content=imported_data[j][0]
        answers=get_answers(imported_data[j][1])
        question=Question(question_type=quez_type,question=question_content)
        
session.close()

[OrderedDict([('题目',
               '彬彬以前的学习成绩一直很好，但自从课程难度加大后，不管彬彬怎样努力学习，总是达不到自己的预期的分数，成绩排名随之下滑。\n如果你遇到和彬彬一样的情况，在成绩长时间下滑时，你最可能有以下哪种想法？'),
              ('选项',
               '0\n一切困难都是可以解决的，我可以尝试改变学习方法或寻求他人帮助\n1\n我担心自己的成绩和未来，需要加倍努力\n1\n成绩有波动也是正常的，我已经尽了自己最大的努力\n2\n我不如以前那么优秀了，提高成绩对于我来说太难了，不知道怎么办\n3\n我以前的“优秀”都是假的，现在努力也是徒劳，未来变得黯淡无光')]),
 OrderedDict([('题目',
               '果果从小学习成绩一直不太理想，被贴上了“差生”的标签。新学期开始了，果果想要好好学习，但仍没有取得实质进展。\n如果你遇到和果果一样的情况，在新学期学习时，你最可能有以下哪种行为？'),
              ('选项',
               '0\n调整心态，设立目标，尝试多种学习方法来挖掘自己的潜力\n1\n不由自主地感到自卑，总是呆在角落，默默努力，做个小透明\n1\n机械地按照学校的节奏学习和生活，完成必要的学习任务\n2\n无法集中精力学习，容易情绪失控，通过娱乐来逃避学习\n3\n不相信自己还可以取得进步，放弃学习，也不再跟老师和同学联系')]),
 OrderedDict([('题目',
               '成成是学校里的尖子生，热爱思考和挑战，总是为目标全力以赴。但在升学考试中，成成没有考入自己理想的学校。\n如果你遇到和成成一样的情况，在考入不是自己理想的学校时，你最可能有以下哪种想法？'),
              ('选项',
               '0\n我相信自己，是金子总会发光，我会增进知识技能，为未来打下坚实的基础\n1\n我感到非常失落，但我相信这所学校也有优势和机遇\n1\n人生不如意十之八九，想太多也改变不了现状，不如顺其自然\n2\n我已经在这个水平了，何必像以前那样努力\n3\n我活得好失败，没有能力再去实现理想了')]),
 OrderedDict([('

In [24]:
len(imported_data)

40